In [1]:
import numpy as np
import time

In [4]:
# Import 2QAN compiler passes
from py2qan import BenchArch
from py2qan import HeuristicMapper
from py2qan import QuRouter

ModuleNotFoundError: No module named 'bench_arch'

In [3]:
# Import qiskit 
import qiskit
from qiskit import transpile, QuantumCircuit

## Run 2QAN compiler passes

In [20]:
def qs_compiler(qasm, coupling_map, qaoa=True, layers=1, trials=1, mapper='qap', bgate='rzz', params=None):
    qs_circ = None
    qs_swap = (0, 0) # the number of swaps in the format (#swaps,#swaps merged with circuit gate)
    qs_g2 = 0 # the number of two-qubit gates without decomposition
    # Perform qubit mapping, routing, and scheduling only, without gate decomposition
    for trial in range(trials):
        # Both QAP and Qiskit mappers output inital qubit maps randomly, 
        # one can run the mapper several times to achieve better compilation results
        # Initial qubit mapping 
        start = time.time()
        hmapper = HeuristicMapper(qasm, coupling_map=coupling_map)
        if mapper == 'qap':
            # The default mapper based on Quadratic Assignment Problem
            init_map, cost = hmapper.run_qap(num_iter=200, lst_len=20)
        elif mapper == 'qiskit':
            # The mapper in Qiskit
            init_map = hmapper.run_qiskit(max_iterations=5)
        end = time.time()
        print("Mapper run time: ", end - start)
        # init_map = {circuit qubit index:device qubit index}
        print('The initial qubit map is \n', init_map)

        # Routing and scheduling, takes init_map as input
        router = QuRouter(qasm, init_map=init_map, coupling_map=coupling_map)
        if qaoa:
            # For QAOA, different layers have different gate parameters
            qs_circ0, swaps1 = router.run_qaoa(layers=layers, gammas=params[layers-1][:layers], betas=params[layers-1][layers:], msmt=True) 
        else:
            # For quantum simulation circuits, we assume each layer has the same time steps
            qs_circ0, swaps1 = router.run(layers=layers, msmt='True')
        # qs_circ0 is the routed circuit without gate decomposition
        # swaps1 is a tuple=(#swaps,#swaps merged with circuit gate)

        # Two-qubit gate count and swap count
        qs_circ1 = transpile(qs_circ0, basis_gates=None, optimization_level=3)
        g2_count1 = 0
        if bgate in qs_circ1.count_ops():
            g2_count1 += qs_circ1.count_ops()[bgate]
        if 'unitary' in qs_circ1.count_ops():
            g2_count1 += qs_circ1.count_ops()['unitary']
        if 'swap' in qs_circ1.count_ops():
            g2_count1 += qs_circ1.count_ops()['swap']
        if trial == 0:
            qs_circ = qs_circ1
            qs_swap = swaps1
            qs_g2 = g2_count1 
        elif g2_count1 < qs_g2:
            qs_circ = qs_circ1
            qs_swap = swaps1
            qs_g2 = g2_count1
        print(g2_count1, qs_swap)
    return qs_circ, qs_swap, qs_g2

def qiskit_decompose(circ, basis_gates=['id', 'rz', 'u3', 'u2', 'cx', 'reset'], bgate='cx'):
    # Perform gate decomposition and optimization into cx gate set
    # For decomposition into other gate sets, e.g., the SYC, sqrt iSWAP, iSWAP, 
    # one can use Google Cirq for decomposition or the NuOp (https://github.com/prakashmurali/NuOp) decomposer
    decom_g2 = 0
    decom_circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)
    if bgate in decom_circ.count_ops():
        decom_g2 += decom_circ.count_ops()[bgate] 
    if 'unitary' in decom_circ.count_ops().keys():
        decom_g2 += decom_circ.count_ops()['unitary']
    return decom_circ, decom_g2

## Tests

In [30]:
import os
import pickle as pkl
# Benchmarks
qaoa = True
# QAOA benchmarks
# OpenQASM circuits here only contain one layer/depth
with open(os.path.join('qaoa_qasms.pkl'), 'rb') as f:
    qasms = pkl.load(f)
# The parameters here include gammas for rzz and betas for rx in 4 layers
with open(os.path.join('qaoa_params.pkl'), 'rb') as f:
    params = pkl.load(f)
    
param = None
idx = -2  # circuit id
c_qasm = qasms[idx]
if qaoa:
    param = params[idx]
test_circ = qiskit.QuantumCircuit.from_qasm_str(c_qasm)

In [6]:
# Device information
# gate set, assume cx as the native two-qubit gate
basis_gates = ['id', 'rz', 'u3', 'u2', 'cx', 'reset']

# topology, assume grid architecture as an example
qn = len(test_circ.qubits)
dx = int(np.sqrt(qn))
print('The number of qubits is ', qn)
if dx*dx >= qn:
    lattice_xy = (dx, dx)
elif dx*(dx+1) >= qn:
    lattice_xy = (dx, dx+1)
elif dx*(dx+2) >= qn:
    lattice_xy = (dx, dx+2)
grid_topology = BenchArch(c_qasm, lattice_xy=lattice_xy).topology
coupling_map = [list(edge) for edge in list(grid_topology.edges)]
coupling_map += [[edge[1], edge[0]] for edge in list(grid_topology.edges)]

The number of qubits is  20


## QAP mapper + 2QAN routing&scheduling

In [24]:
qs_circ, qs_swap, qs_g2 = qs_compiler(c_qasm, coupling_map, qaoa=qaoa, layers=1, trials=5, bgate='rzz', params=param)
print('The number of SWAPs: ', qs_swap, qs_g2)

Mapper run time:  13.515033960342407
The initial qubit map is 
 {0: 9, 1: 18, 2: 4, 3: 17, 4: 1, 5: 16, 6: 2, 7: 3, 8: 15, 9: 11, 10: 0, 11: 19, 12: 7, 13: 10, 14: 12, 15: 13, 16: 6, 17: 14, 18: 5, 19: 8}
34 (11, 7)
Mapper run time:  15.803452968597412
The initial qubit map is 
 {0: 10, 1: 12, 2: 1, 3: 8, 4: 3, 5: 4, 6: 11, 7: 15, 8: 17, 9: 19, 10: 2, 11: 16, 12: 14, 13: 18, 14: 5, 15: 9, 16: 7, 17: 13, 18: 6, 19: 0}
33 (8, 5)
Mapper run time:  14.26235556602478
The initial qubit map is 
 {0: 15, 1: 6, 2: 5, 3: 2, 4: 8, 5: 0, 6: 16, 7: 17, 8: 11, 9: 18, 10: 9, 11: 10, 12: 14, 13: 19, 14: 1, 15: 3, 16: 13, 17: 7, 18: 12, 19: 4}
37 (8, 5)
Mapper run time:  13.522371053695679
The initial qubit map is 
 {0: 8, 1: 14, 2: 15, 3: 6, 4: 16, 5: 7, 6: 17, 7: 13, 8: 1, 9: 9, 10: 19, 11: 10, 12: 5, 13: 4, 14: 3, 15: 2, 16: 18, 17: 0, 18: 12, 19: 11}
34 (8, 5)
Mapper run time:  13.557611227035522
The initial qubit map is 
 {0: 7, 1: 4, 2: 17, 3: 8, 4: 18, 5: 12, 6: 14, 7: 10, 8: 1, 9: 6, 10: 19, 11

In [27]:
qs_circ2, qs_g2 = qiskit_decompose(qs_circ, bgate='cx', basis_gates=basis_gates)
print('The number of CNOTs: ', qs_g2)

The number of CNOTs:  74


## Qiskit SABRE mapper + 2QAN routing&scheduling

In [25]:
ibm_circ, ibm_swap, ibm_g2 = qs_compiler(c_qasm, coupling_map, qaoa=qaoa, layers=1, trials=5, mapper='qiskit', bgate='rzz', params=param)
print('The number of SWAPs: ', ibm_swap, ibm_g2)

Mapper run time:  0.30110716819763184
The initial qubit map is 
 {0: 8, 1: 10, 2: 11, 3: 0, 4: 12, 5: 7, 6: 13, 7: 16, 8: 5, 9: 14, 10: 15, 11: 6, 12: 9, 13: 17, 14: 2, 15: 1, 16: 19, 17: 4, 18: 18, 19: 3}
37 (15, 8)
Mapper run time:  0.21735572814941406
The initial qubit map is 
 {0: 7, 1: 12, 2: 16, 3: 14, 4: 4, 5: 17, 6: 8, 7: 9, 8: 10, 9: 2, 10: 13, 11: 5, 12: 6, 13: 3, 14: 19, 15: 15, 16: 0, 17: 11, 18: 1, 19: 18}
36 (9, 3)
Mapper run time:  0.21683311462402344
The initial qubit map is 
 {0: 13, 1: 14, 2: 18, 3: 12, 4: 11, 5: 9, 6: 7, 7: 3, 8: 5, 9: 1, 10: 15, 11: 0, 12: 4, 13: 2, 14: 17, 15: 16, 16: 10, 17: 8, 18: 6, 19: 19}
39 (9, 3)
Mapper run time:  0.24787020683288574
The initial qubit map is 
 {0: 18, 1: 5, 2: 4, 3: 13, 4: 15, 5: 8, 6: 14, 7: 6, 8: 10, 9: 1, 10: 3, 11: 0, 12: 9, 13: 2, 14: 16, 15: 17, 16: 7, 17: 19, 18: 11, 19: 12}
41 (9, 3)
Mapper run time:  0.2088031768798828
The initial qubit map is 
 {0: 10, 1: 6, 2: 7, 3: 4, 4: 19, 5: 1, 6: 18, 7: 17, 8: 13, 9: 12, 10: 

In [26]:
ibm_circ2, ibm_g2 = qiskit_decompose(ibm_circ, bgate='cx', basis_gates=basis_gates)
print('The number of CNOTs: ', ibm_g2)

The number of CNOTs:  78


## Qiskit compiler

In [28]:
# test_circ only has one layer in the given example
qiskit_circ = transpile(test_circ, basis_gates=basis_gates, coupling_map=coupling_map, optimization_level=3)
print('The number of CNOTs: ', qiskit_circ.count_ops()['cx'])

The number of CNOTs:  94
